In [ ]:
from pathlib import Path

import mlflow
import pandas as pd
import seaborn as sns
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error

sns.set_theme()
pd.options.mode.copy_on_write = True

TRAIN_DATA_FILE_PATH = '../data/green_tripdata_2021-01.parquet'
VAL_DATA_FILE_PATH = '../data/green_tripdata_2021-02.parquet'

In [ ]:
# Setting backend store URI and artifact location
home_path_str = Path.home()
backend_store_uri = f'sqlite:///{home_path_str}/mlflow/backend/mlflow.db'
artifact_location = f'file:{home_path_str}/mlflow/mlruns'

# Setting backend store
mlflow.set_tracking_uri(uri=backend_store_uri)

# Setting experiment name
experiment_name = "green-taxi-nyc-trip-duration"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    experiment_id = mlflow.create_experiment(
        name=experiment_name,
        artifact_location=artifact_location
    )
mlflow.set_experiment(experiment_name=experiment_name)

In [ ]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        green_taxi_trips = pd.read_csv(filename)

        green_taxi_trips['lpep_dropoff_datetime'] = pd.to_datetime(green_taxi_trips['lpep_dropoff_datetime'])
        green_taxi_trips['lpep_pickup_datetime'] = pd.to_datetime(green_taxi_trips['lpep_pickup_datetime'])
    elif filename.endswith('.parquet'):
        green_taxi_trips = pd.read_parquet(filename)

    green_taxi_trips['duration'] = green_taxi_trips['lpep_dropoff_datetime'] - green_taxi_trips['lpep_pickup_datetime']
    green_taxi_trips['duration'] = green_taxi_trips['duration'].apply(lambda td: td.total_seconds() / 60)

    green_taxi_trips = green_taxi_trips[(green_taxi_trips['duration'] >= 1) & (green_taxi_trips['duration'] <= 60)]

    categorical_vars = ['PULocationID', 'DOLocationID']
    green_taxi_trips[categorical_vars] = green_taxi_trips[categorical_vars].astype(str)

    return green_taxi_trips


df_train = read_dataframe(TRAIN_DATA_FILE_PATH)
df_val = read_dataframe(VAL_DATA_FILE_PATH)

In [ ]:
categorical = ['PU_DO']  #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [ ]:
best_params = {
    'learning_rate': 0.09585355369315604,
    'max_depth': 30,
    'min_child_weight': 1.060597050922164,
    'objective': 'reg:linear',
    'reg_alpha': 0.018060244040060163,
    'reg_lambda': 0.011658731377413597,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)